### EDA - auto+mpg - Parte II

In [ ]:
# Exemplo EDA sobre o dataset de datos de coches
# https://www.datafied.world/eda-on-mpg-data-using-seaborn-192

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 'mpg' é un coñecido dataset con datos de automóbiles que se encontra como exemplo na libraría 'seaborn'
# Tamén dispoñible no UCI: Machine Learning Repository
# https://archive.ics.uci.edu/ml/datasets/auto+mpg

# O repositorio proporciona a seguinte información:
# Attribute Information:

# 1. mpg: continuous
# 2. cylinders: multi-valued discrete
# 3. displacement: continuous
# 4. horsepower: continuous
# 5. weight: continuous
# 6. acceleration: continuous
# 7. model year: multi-valued discrete
# 8. origin: multi-valued discrete
# 9. car name: string (unique for each instance)

In [ ]:
df = sns.load_dataset('mpg')

In [ ]:
# Fases do EDA
# 1. Preprocesar os datos
# 2. EDA sobre atributos categóricos: analizar distribución e relacións con outros categóricos
# 3. EDA sobre atributos numéricos: analizar distribució e relacións con outros numéricos
# 4. Análise da relación entre categóricos e numéricos

In [ ]:
# Nesta parte II centrámonos na análise dos atributos numéricos

In [ ]:
# Cargamos o dataset gardado despois da parte do EDA de variables categóricas
df = pd.read_csv("mpg_cated.csv")
df.head()

In [ ]:
# Na primeira parte definimos cales eran as variables numéricas na variable 'nums'
nums = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']
print(f'numerical variables:  {nums}')

In [ ]:
# As variables están ordenadas no datafram, coas variables numéricas na parte dereita
# Podo crear un novo dataframe unicamente para os datos das variables numéricas
df_num = df.loc[:, 'mpg':]

In [ ]:
df_num.describe()

#### Análise de Distribución dos atributos numéricos

In [ ]:
rows = len(nums)
cols = 3

fig = plt.figure(1, (18, rows*3))

i = 0
for col in nums:
    
    i += 1
    ax1 = plt.subplot(rows, cols,i)
#     ax1.hist(df[col], alpha=0.6)
    sns.distplot(df_num[col])
    ax1.set_xlabel(None)
    ax1.set_title(f'{col} distribution')
    plt.tight_layout()

    i += 1
    ax2 = plt.subplot(rows, cols,i)
    sns.violinplot(df_num[col])
    sns.swarmplot(df_num[col], alpha=0.6, color='k')
    ax2.set_xlabel(None)
    ax2.set_title(f'{col} swarm-violin plot')
    plt.tight_layout()

    i += 1
    ax3 = plt.subplot(rows, cols,i)
    sns.boxplot(df_num[col], orient='h', linewidth=2.5)
    ax3.set_xlabel(None)
    ax3.set_title(f'{col} box plot')
    plt.tight_layout()



**Insights**
- 'acceleration' é a única distribución que se axusta á 'Normal' (ou distribución gaussiana) (con algúns outliers)
- 'mpg' e 'weight' teñen distribucións que semellan normais, mais sesgadas á dereita, é dicir, concentran maior número de ocorrencias na parte esquerda da "campana"
- 'displacement' e 'horsepower' semellan lonxe da  normal

##### Análise dos outliers

In [ ]:
# Utilizando a formula de tukey

def tukey_outliers(x):
    q1 = np.percentile(x,25)
    q3 = np.percentile(x,75)
    
    iqr = q3-q1
    
    min_range = q1 - iqr*1.5
    max_range = q3 + iqr*1.5
    
    outliers = x[(x<min_range) | (x>max_range)]
    return outliers

for col in nums:
    outliers = tukey_outliers(df_num[col])
    if len(outliers):
        print(f"* {col} has these tukey outliers,\n{outliers}\n")
    else:
        print(f"* {col} doesn't have any tukey outliers.\n")

In [ ]:
# Mostra os outliers na columna de 'acceleration'

df.iloc[list(tukey_outliers(df_num.acceleration).index)]

In [ ]:
# Mostra os outliers na columna de 'horsepower'

df.iloc[list(tukey_outliers(df_num.horsepower).index)]

**Insights**
- Os outliers en 'acceleration' parecen ser aleatorios, non se detecta ningún patrón.
- Os outliers en 'horsepower' non parecen random, teñen cousas en común:
  - orixe: todos son de usa
  - cylinders: todos teñen 8 cilindros
  - mpg_level: todos teñen un low mpg_leve -> moito consumo
  - weight: practicamente todos superan os 4000 

  - model_year: trátase de modelos dos mais vellos do dataset (inicios dos 70')

#### Relación entre os diferentes atributos numéricos

In [ ]:
# Pairplot, de Seaborn, encárgase de crear unha matriz de gráficas cruzando as variables que indiquemos
# a través de scatterplots.
# Pairplot utiliza a diagonal para pintar unha gráfica de distribución de cada unha das variables
# Fíxate na simetría dos gráficos da esquina inferior esquerda e superior directa, pois son o resultado
# de mudar os eixos X e Y

# Info sobre pairplots
# https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166

In [ ]:
sns.pairplot(data=df, vars=nums, diag_kind='kde', hue='origin',
            plot_kws=dict(s=20, edgecolor="k", linewidth=0.1, alpha=0.5), diag_kws=dict(shade=True))
plt.show()

In [ ]:
# Visualizamos unha matriz de correlación entre as diferentes variables numéricas, o que nos da unha
# rápida impresión das correlacións máis fortes (cores escuros)

In [ ]:
sns.heatmap(df_num.corr(method='spearman'), annot = True, vmin = -1, vmax = 1, center = 0, cmap = 'coolwarm')
plt.show()

**Insights**
- A medida que aumenta o mpg (descende o consumo) o 'displacement', 'horsepower' e 'weight' diminúen, pero acceleración aumenta
- O incremento no 'horsepower' aumenta 'diplacement' e 'weight' pero decrementa 'acceleration'
- 'Weight' aumenta con 'displacement' pero decrementa 'acceleration'
- A medida que aumenta 'acceleration' descende 'displacement'

Todos as variables correlacionan, en maior ou menor medida, coas demais

In [ ]:
# Algúns gráficos permiten relacionar variables numéricas engadindos diferentes categorías de xeito simultáneo
# Pode utilizarse a cor, o tamaño, ou debuxar varias gráficas atendendo aos valores da categoría

In [ ]:
'''In this plot we analyze the relationship of horsepower & acceleration
bifurcated by origin, mpg_level & cylinders in a single plot.'''

sns.relplot(x='horsepower', y='acceleration', hue='mpg_level', #style='mpg_level',
            size='cylinders', col='origin', data=df, kind='scatter', sizes=(5, 100), alpha=0.6)
plt.show()

**Insights**
- En todas as rexións hai unha correlación negativa entre horsepower e acceleration
- Vehículos con low "mpg_level" teñen maior horsepower e menor acceleration
- Vehículos con máis cilindros teñen maior horsepower e menos acceleration

In [ ]:
'''In this plot we analyze the relationship of weight & horsepower
bifurcated by origin, mpg_level & cylinders in a single plot.'''

sns.relplot(x='weight', y='horsepower', hue='mpg_level', #style='mpg_level',
            size='cylinders', col='origin', data=df, kind='scatter', sizes=(5, 100), alpha=0.6)
plt.show()

**Insights**
- En todas as rexións hai unha correlación positiva entre peso e potencia (weight e horsepower)
- Vehículos con baixo mpg teñen mais peso e potencia (weight e horsepower)
- Vehículos con máis cilindros teñen mais peso e potencia (weight e horsepower)